# Hospitalization and Wastewater Data Retrieval and Visualization
#         People's CDC   http://peoplescdc.org
#               Updated 12/30/2023 @10:55 AM  by Les Schaffer

## Setup: be sure to run these first two cells

In [ ]:
import os
import re
import sys

try:
    from google.colab import userdata, output
    output.enable_custom_widget_manager()
    api_key_id = userdata.get('api_key_id')
    api_key_secret = userdata.get('api_key_secret')
    api_app_token = userdata.get('api_app_token')
    ! pip install sodapy
    ! pip install ipympl
except ModuleNotFoundError:
    sys.path.append(os.path.abspath('../'))
    from tokens import api_key_id, api_key_secret, api_app_token

get_ipython().run_line_magic('matplotlib', 'ipympl')

from sodapy import Socrata
from pandas import DataFrame, to_datetime, Series, concat, isna
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import seaborn as sbrn

# set up Socrata clients for data queries
# hospital data by facility and by state
URL = 'healthdata.gov'
hclient = Socrata(URL, api_app_token,  username=api_key_id,   password=api_key_secret, timeout=20 )

# wastewater data by facility
URL = 'data.cdc.gov'
wclient = Socrata(URL, api_app_token,  username=api_key_id,   password=api_key_secret, timeout=20 )

In [ ]:
def format_axis(ax=None):
    _num2date = mdates.num2date

    def formatter(x, y):
        the_date = _num2date(x).date().isoformat()
        return f'x={the_date}, y={y:.4g}'

    if ax is None:
        ax = plt.gca()
    ax.format_coord = formatter


# Hospitalization -- by facility

## metadata

In [ ]:
RSRC ='anag-cw7u'
metadata = hclient.get_metadata(RSRC)
print([x['name'] for x in metadata['columns']])
print(metadata.keys())
print(metadata['rowsUpdatedBy'])
print(metadata['owner'])
metadata['metadata']
metadata['columns']

meta_amount = [x for x in metadata['columns'] if x['name'] == 'inpatient_beds_used_covid_7_day_avg']
meta_amount

## get the data

In [ ]:
RSRC ='anag-cw7u'
# Use the 'where' argument to filter the data before downloading it
results = hclient.get(RSRC, where=u"collection_week >= '2023-12-11T00:00:00.000'", limit=100000)   # AND hospital_subtype = 'Children's Hospitals'")
results=DataFrame.from_records(results)
print("Number of results downloaded: {}".format(len(results)))

results['collection_week'] = to_datetime(results['collection_week'])
print(results['collection_week'].sort_values().unique())

## make tables and plots
### Children's hospitals: covid bed fraction > 20%

In [ ]:
query = """
SELECT
collection_week,
hospital_name,
all_pediatric_inpatient_beds_7_day_avg,
total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,
inpatient_beds_7_day_avg
WHERE
collection_week>'2020-03-01T00:00:00.000'
AND hospital_subtype = "Childrens Hospitals"
ORDER BY
collection_week
LIMIT
40000
"""
RSRC ='anag-cw7u'
results = hclient.get(RSRC, query=query)
res=DataFrame.from_records(results)
res=res.replace('-999999.0', '2.0')
res['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg'] = res['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg'].astype(float)
res['all_pediatric_inpatient_beds_7_day_avg']=res['all_pediatric_inpatient_beds_7_day_avg'].astype(float)
res['inpatient_beds_7_day_avg'] = res['inpatient_beds_7_day_avg'].astype(float)
res['percent'] = 100*res['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg']/res['all_pediatric_inpatient_beds_7_day_avg']
# cases where all_pediatric_inpatient_beds_7_day_avg is reported as 0!!!! then use inpatient_beds_7_day_avg
use_inpatient_beds = res['all_pediatric_inpatient_beds_7_day_avg']==0.0
res.loc[use_inpatient_beds, 'percent'] = 100*res[use_inpatient_beds]['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg']/res[use_inpatient_beds]['inpatient_beds_7_day_avg']

num = res.loc[:,'percent'].count()
print(f'Number of times facilities reporting = {num}')
hidden = res['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg']==2.0
num = res.loc[hidden,'percent'].count()
print(f'Number of times facilities reporting -999999 --> 2 = {num}')
plt.figure()
res.loc[~hidden, 'percent'].plot(kind='hist', bins=100, logy=True)
plt.gcf().suptitle('# reports of percent usage')
plt.gca().set_xlabel('percent usage')
res[res['percent']>=20].sort_values('percent', ascending=False)

### lets look at average usage over the entire pandemic > 5% !!!

In [ ]:
df=res.groupby(['hospital_name',])['percent'].mean()
df[df>=5.]

### Pediatric admission, nationwide

In [ ]:
query = """
SELECT
previous_day_admission_pediatric_covid_confirmed_7_day_sum,
collection_week
WHERE
collection_week>'2020-01-03T00:00:00.000' AND hospital_subtype = "Childrens Hospitals"
ORDER BY
collection_week ASC
LIMIT
30000
"""
RSRC ='anag-cw7u'
results = hclient.get(RSRC, query=query)
res=DataFrame.from_records(results)
res=res.replace('-999999', '2')
res['collection_week'] = to_datetime(res['collection_week'])
res=res[res['previous_day_admission_pediatric_covid_confirmed_7_day_sum'].notna()]
res['previous_day_admission_pediatric_covid_confirmed_7_day_sum']=res['previous_day_admission_pediatric_covid_confirmed_7_day_sum'].astype(int)
df=res.groupby('collection_week').sum()
plt.figure()
plt.plot(df['previous_day_admission_pediatric_covid_confirmed_7_day_sum'])
plt.tick_params('x', rotation=90)
plt.title('pediatric admissions -- 7-day sums')
res

## Adults

In [ ]:
query = """
SELECT
previous_day_admission_adult_covid_confirmed_7_day_sum,
total_adult_patients_hospitalized_confirmed_covid_7_day_avg,
total_adult_patients_hospitalized_confirmed_covid_7_day_sum,
collection_week
WHERE
collection_week>'2020-01-03T00:00:00.000' AND hospital_subtype != "Childrens Hospitals"
ORDER BY
collection_week ASC
LIMIT
1000000
"""
RSRC ='anag-cw7u'
results = hclient.get(RSRC, query=query)
ares=DataFrame.from_records(results)
ares=ares.replace('-999999', '2')
ares=ares.replace('-999999.0', '2.0')
ares['collection_week'] = to_datetime(ares['collection_week'])

ares=ares[ares['previous_day_admission_adult_covid_confirmed_7_day_sum'].notna()]
ares['previous_day_admission_adult_covid_confirmed_7_day_sum']=ares['previous_day_admission_adult_covid_confirmed_7_day_sum'].astype(int)

ares=ares[ares['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'].notna()]
ares['total_adult_patients_hospitalized_confirmed_covid_7_day_avg']=ares['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'].astype(float)

ares=ares[ares['total_adult_patients_hospitalized_confirmed_covid_7_day_sum'].notna()]
ares['total_adult_patients_hospitalized_confirmed_covid_7_day_sum']=ares['total_adult_patients_hospitalized_confirmed_covid_7_day_sum'].astype(float)
ares

In [ ]:
df=ares.groupby('collection_week').sum()
fig, (ax1,ax2) = plt.subplots(2,1, sharex=True)
ax1.plot(df['previous_day_admission_adult_covid_confirmed_7_day_sum']/7)
ax1.tick_params('x', rotation=90)
ax1.set_title('adult admissions -- 7-day ave')

ax2.plot(df['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'])
ax2.tick_params('x', rotation=90)
ax2.set_title('adult hospitalized -- 7-day ave')

# by state and nation, adult
## metadata first, only if needed

In [ ]:
RSRC ='g62h-syeh'
metadata = hclient.get_metadata(RSRC)

# print([x['name'] for x in metadata['columns']])
print(metadata.keys())
print(metadata['rowsUpdatedBy'])
print(metadata['owner'])
metadata['metadata']
metadata['columns']

## get the data

In [ ]:
RSRC ='g62h-syeh'
query = """
SELECT
hospital_onset_covid,
deaths_covid,
previous_day_admission_adult_covid_confirmed,
percent_of_inpatients_with_covid,
state,
date
WHERE
date>'2020-01-03T00:00:00.000'
ORDER BY
date ASC
LIMIT
100000
"""
colL = ('hospital_onset_covid', 'deaths_covid', 'previous_day_admission_adult_covid_confirmed', 'percent_of_inpatients_with_covid')
results = hclient.get(RSRC, query=query)
hsres=DataFrame.from_records(results)
hsres.fillna(0,inplace=True)
hsres['date'] = to_datetime(hsres['date'])
for colN in colL[:-1]:
    hsres[colN] = hsres[colN].astype(int)

hsres[colL[-1]] = hsres[colL[-1]].astype(float)
print(hsres.dtypes)
print(hsres)
hs_col_order = hsres['state'].unique().tolist()
hs_col_order.sort()

## Visualize the data sets
### needs 7-day rolling averages
### needs better handling of labels (shorten the column names)
### needs ScatterMapBox maps for by_facility

### Nationwide

In [ ]:
fig,axL = plt.subplots(3,1, sharex=True, figsize=(12,8))
axL[0].set_title('Nationwide')
for stat, ax in zip( ('deaths_covid','hospital_onset_covid',  'previous_day_admission_adult_covid_confirmed'), axL):
    national = hsres.groupby('date')[stat].sum()
    ax.plot_date(national.index,national,ms=1)
    ax.tick_params('x', rotation=90)
    ax.set_ylabel(stat)
    format_axis(ax)

### hospital_onset_covid

In [ ]:
graph1 = sbrn.FacetGrid(hsres, col='state', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=8)
graph1.map(plt.plot_date, 'date', 'hospital_onset_covid', ms=1)
graph1.tick_params('x', rotation=90)
graph1.savefig('hospital_onset_covid.png')

### deaths_covid

In [ ]:
graph2 = sbrn.FacetGrid(hsres, col='state', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=8)
graph2.map(plt.plot_date, 'date', 'deaths_covid', ms=1)
graph2.tick_params('x', rotation=90)
graph2.savefig('deaths_covid.png')

### previous_day_admission_adult_covid_confirmed

In [ ]:
graph3 = sbrn.FacetGrid(hsres, col='state', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=8)
graph3.map(plt.plot_date, 'date', 'previous_day_admission_adult_covid_confirmed', ms=1)
graph3.tick_params('x', rotation=90)
graph3.savefig('previous_day_admission_adult_covid_confirmed.png')

### percent_of_inpatients_with_covid

In [ ]:
graph4 = sbrn.FacetGrid(hsres, col='state', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=8)
graph4.map(plt.plot_date, 'date', 'percent_of_inpatients_with_covid', ms=1)
graph4.tick_params('x', rotation=90)
graph4.savefig('percent_of_inpatients_with_covid.png')

## data summary over pandemic, by state

In [ ]:
dataD = {}
totalD = {'state':'TOTAL/AVE'}
for colN in colL[:-1]:
    dataD[colN]=hsres.groupby(['state', ])[colN].sum()
    totalD[colN] = hsres[colN].sum()

dataD[colL[-1]] = hsres.groupby(['state', ])[colL[-1]].mean()
dataD['max percentage'] = hsres.groupby(['state', ])[colL[-1]].max()

totalD[colL[-1]] = hsres[colL[-1]].mean()
totalD['max percentage'] = dataD['max percentage'].mean()
df1=DataFrame(dataD)
df2=DataFrame(totalD, index=['last'])
df2=df2.set_index('state')
df = concat([df1,df2])
df.to_csv('state-summary.csv')
df


# Wastewater code
### need to handle visualization maps by-facility
### need to develop code to bring up to county and state/nation
### N.B. some plants with ONE unique ID are delivering two or more sets of numbers on a given day. e.g.: ma_1495

### initialization: needed functions and build waste treatment plant database

In [ ]:
# some  functions for later use
# x='CDC_BIOBOT_ny_505_Treatment plant_raw wastewater'
#             wwtp_jurisdiction 	wwtp_id 	reporting_jurisdiction 	sample_location 	sample_location_specify 	key_plot_id 	                                    county_names 	county_fips 	population_served
# 252219 	  California 	        2526 	        California 	        Treatment plant 	NaN 	                    CDC_VERILY_ca_2526_Treatment plant_post grit r... 	Ventura 	    06111 	          250000
# 252487 	  Texas 	            2527 	        Texas 	            Treatment plant 	635 	                    CDC_VERILY_tx_2527_Treatment plant_635_raw was... 	Travis 	        48453 	          529541
# 252884 	  California 	        834 	        California 	        Treatment plant 	633 	                    CDC_VERILY_ca_834_Treatment plant_633_post gri... 	Riverside 	    06065 	           91980
regex = re.compile('_[a-z][a-z]_(\d+)_')

def get_id(row):
    x = row.key_plot_id
    y = regex.search(x)
    if y:
        shortened = x[0:y.end()-1] 
    else:
        print(row)
        shortened = x.split(' ')[0]
        print('id w/o #: ', shortened)
    # NWSS_az_1212 1212
    if not isna(row.sample_location_specify):
        shortened = shortened + f'_{row.sample_location_specify}'
    elif x.find('post grit')>-1:
        shortened=shortened+'_post'
    return shortened

def get_col_order(df, key_column):
    col_order = df[key_column].unique().tolist()
    col_order.sort()
    return col_order

def update_wwtp_id(df):
    df['wwtp_id'] = df.apply(get_id, axis=1)
    return df

In [ ]:
fieldStr = 'wwtp_jurisdiction,wwtp_id,reporting_jurisdiction,sample_location,sample_location_specify,key_plot_id,county_names,county_fips,population_served'
query = f"""
SELECT
{fieldStr}
WHERE
date_end>'2023-07-01T00:00:00.000'
LIMIT
300000
"""

        
results = wclient.get("2ew6-ywp6", query=query)
res0=DataFrame.from_records(results)

print(res0['key_plot_id'].count())

grp=res0.groupby('key_plot_id').nth(0)
grp = grp.astype({'population_served': int, 'sample_location_specify': 'Int64' })
grp = update_wwtp_id(grp)
grp.to_csv('wwtp_db.csv')
mults = grp.groupby('wwtp_id',group_keys=False).apply(lambda x: x if x.key_plot_id.count()>1 else None) 
print(mults)
mults.to_csv('multiples.csv')
grp

## by facility. can get all facilities in a state by inputting two letter state abbrev or particular facility by state+wwtp_id+sample_location_specify if existing
### enter state abbreviation at prompt

In [ ]:
wstate = input('enter state abbreviation for displaying wastewater results: ')

## By concentration
## https://data.cdc.gov/Public-Health-Surveillance/NWSS-Public-SARS-CoV-2-Concentration-in-Wastewater/g653-rqe2/about_data 
## pcr_conc_smoothed

In [ ]:
RSRC ='g653-rqe2'

query = f"""
SELECT
*
WHERE
date>='2020-01-03T00:00:00.000' AND key_plot_id LIKE '%\\_{wstate}\\_%'
ORDER BY
date ASC
LIMIT
90000
"""

print(query)
results = wclient.get(RSRC, query=query)
res1=DataFrame.from_records(results)
res1['pcr_conc_smoothed'] =  res1['pcr_conc_smoothed'].astype(float)
res1['date'] = to_datetime(res1['date'])
res1 = res1.join(grp.set_index('key_plot_id'), on='key_plot_id')
w1_col_order=get_col_order(res1, 'wwtp_id')
# need to groupby:
res1['percentile_from_conc'] = res1.groupby('key_plot_id', group_keys=False).apply(lambda x: 100*x.pcr_conc_smoothed/ x.pcr_conc_smoothed.max())
res1.to_csv(f'{wstate}_wastewater.csv')
res1

# why negative numbers for pcr_conc_smoothed???
#     key_plot_id   date      pcr_conc_smoothed    normalization       wwtp_id
# 57          527 2021-10-15      -2.875224e+06  flow-population   NWSS_ny_527
# 58           52 2021-10-15       2.960513e+07  flow-population    NWSS_ny_52
# 59         2182 2021-10-15       5.964828e+05  flow-population  NWSS_ny_2182
# 60          426 2021-10-15       1.424451e+07  flow-population   NWSS_ny_426
# 61          515 2021-10-15      -1.071621e+07  flow-population   NWSS_ny_515
# 62          522 2021-10-15      -4.880599e+06  flow-population   NWSS_ny_522
# 63          524 2021-10-15      -6.443013e+06  flow-population   NWSS_ny_524
# 64          529 2021-10-15      -1.401466e+07  flow-population   NWSS_ny_529

In [ ]:
plt.figure(figsize=(14,8))
Normalize=False
if Normalize:
    res1.groupby('key_plot_id').apply(lambda x: plt.plot_date(x.date, x.pcr_conc_smoothed/x.pcr_conc_smoothed.max(), '.', ms=1, label=x.wwtp_id.iloc[0]))
    plt.ylim(0.,1)
    plt.ylabel('percentile concentration, normalized')
else:
    res1.groupby('key_plot_id').apply(lambda x: plt.plot_date(x.date, x.pcr_conc_smoothed, '.', ms=1, label=x.wwtp_id.iloc[0]))
    plt.ylabel('percentile concentration')
plt.legend(ncols=3)

In [ ]:
graph1 = sbrn.FacetGrid(res1, col='wwtp_id', col_order=w1_col_order, sharex=True, sharey=False, col_wrap=7)
graph1.map(plt.plot_date, 'date', 'pcr_conc_smoothed', ms=1)
graph1.tick_params('x', rotation=90)
graph1.savefig(f'{wstate}-wastewater-1.png')

graph4 = sbrn.FacetGrid(res1, col='wwtp_id', col_order=w1_col_order, sharex=True, sharey=False, col_wrap=7)
graph4.map(plt.plot_date, 'date', 'percentile_from_conc', ms=1)
graph4.tick_params('x', rotation=90)
graph4.savefig(f'{wstate}-wastewater-2a.png')

## Metric data
### https://data.cdc.gov/Public-Health-Surveillance/NWSS-Public-SARS-CoV-2-Wastewater-Metric-Data/2ew6-ywp6/about_data



In [ ]:
fieldStr = 'key_plot_id,date_start,date_end,percentile,ptc_15d'
query = f"""
SELECT
{fieldStr}
WHERE
date_end>'2020-01-01T00:00:00.000' AND key_plot_id LIKE '%\\_{wstate}\\_%'
LIMIT
100000
"""
results = wclient.get("2ew6-ywp6", query=query)
res2=DataFrame.from_records(results)
res2 = res2.join(grp.set_index('key_plot_id'), on='key_plot_id')
res2['percentile'] =  res2['percentile'].astype(float)
res2 = res2.astype({'population_served': int, 'sample_location_specify': 'Int64' })
res2['date_end'] = to_datetime(res2['date_end'])
w2_col_order=get_col_order(res2, 'wwtp_id')
res2.to_csv(f'{wstate}_metric_data_wastewater.csv')
res2

## Percentile
### needs comparison with the pcr_conc_smoothed

### every graph should have at least one 100%. but not.
#### This metric shows whether SARS-CoV-2 virus levels at a site are currently higher or lower than past historical levels at the same site.
#### 0% means levels are the lowest they have been at the site; 100% means levels are the highest they have been at the site.
#### Public health officials watch for increasing levels of the virus in wastewater over time and use this data to help make public health decisions.

In [ ]:
graph2 = sbrn.FacetGrid(res2, col='wwtp_id', col_order=w2_col_order, sharex=True, sharey=False, col_wrap=7)
graph2.map(plt.plot_date, 'date_end', 'percentile', ms=1)
graph2.tick_params('x', rotation=90)
graph2.savefig(f'{wstate}-wastewater-2.png')

### PTC_15D whatever that is, it doesn't LOOK like much. maybe i goofed here???

### quoteth the Law(d):

#### The percent change in SARS-CoV-2 RNA levels over the 15-day interval defined by 'date_start' and 'date_end'.
#### Percent change is calculated as the modeled change over the interval, based on linear regression of log-transformed SARS-CoV-2 levels.
#### SARS-CoV-2 RNA levels are wastewater concentrations that have been normalized for wastewater composition.

In [ ]:
cutoff = float(input('enter upper end cutoff:  '))
res2['ptc_15d'] =  res2['ptc_15d'].astype(float)
ptc_cropped = res2[res2['ptc_15d']<cutoff]

graph3 = sbrn.FacetGrid(ptc_cropped, col='wwtp_id', col_order=w2_col_order, sharex=True, sharey=False, col_wrap=7)
graph3.map(plt.plot_date, 'date_end', 'ptc_15d', ms=1)
graph3.tick_params('x', rotation=90)
graph3.savefig(f'{wstate}-wastewater-3.png')

plt.figure()
ptc_cropped['ptc_15d'].plot(kind='hist', bins=400, logx=False, logy=True)
plt.xlabel('% change 15 days')
plt.title(f'Histogram of weird ptc_15d wastewater metric (>{cutoff} excluded)')

### Let's now look at state and national, and eventually, county

In [ ]:
RSRC ='g653-rqe2'

# date>='2020-01-03T00:00:00.000' AND key_plot_id LIKE '%_{wstate}_%'
# date='2023-12-22T00:00:00.000'
query = f"""
SELECT
key_plot_id,
date,
pcr_conc_smoothed
WHERE
date>='2020-01-01T00:00:00.000'
LIMIT
700000
"""

results = wclient.get(RSRC, query=query)
res3=DataFrame.from_records(results)
res3['pcr_conc_smoothed'] =  res3['pcr_conc_smoothed'].astype(float)
res3['date'] = to_datetime(res3['date'])
res3=res3[res3['pcr_conc_smoothed'].notna()]
res3

In [ ]:

x=res3.groupby('date')['pcr_conc_smoothed'].count()
fig, ax = plt.subplots(1,1, figsize=(10,8))
ax.plot_date(x.index, x, ms=1)
ax.set_ylabel('# stations reporting')
format_axis(ax)
ax.tick_params('x', rotation=90)
fig.suptitle('Number wastewater stations reporting')
plt.savefig('num_waste_stations.png')

### join dataset to the facility database

In [ ]:
full = res3.join(grp.set_index('key_plot_id'), on='key_plot_id')
full.sort_values('date')

In [ ]:
full.dtypes
def national_pop_averaged_conc(x):
    y = x['pcr_conc_smoothed'] * x['population_served']/tot_pop
    return y.sum()

In [ ]:
tot_pop=grp['population_served'].sum()
tot_pop

In [ ]:
ngrp = full.groupby(['date']).apply(lambda x: x['pcr_conc_smoothed'].mean())
plt.figure()
plt.plot_date(ngrp.index, ngrp, ms=1)
plt.ylabel('concentration average')
plt.savefig('wastewater_by_nation.png')
ngrp

In [ ]:
sgrp = full.groupby(['date', 'wwtp_jurisdiction']).apply(lambda x: x['pcr_conc_smoothed'].mean()).reset_index(name='concentration average')
graph_conc_state = sbrn.FacetGrid(sgrp, col='wwtp_jurisdiction', col_order=get_col_order(sgrp, 'wwtp_jurisdiction'), sharex=True, sharey=False, col_wrap=8)
graph_conc_state.map(plt.plot_date, 'date', 'concentration average', ms=1)
graph_conc_state.tick_params('x', rotation=90)
graph_conc_state.savefig('wastewater_by_state.png')

In [ ]:
res3[res3['pcr_conc_smoothed']<0].groupby('key_plot_id').apply(lambda x: x.pcr_conc_smoothed.count()).to_csv('below-zero.csv')

In [ ]:
fieldStr = 'wwtp_jurisdiction,wwtp_id,reporting_jurisdiction,sample_location,sample_location_specify,key_plot_id,county_names,county_fips,population_served,date_end'
query = f"""
SELECT
{fieldStr}
WHERE
date_end>'2020-01-01T00:00:00.000' AND reporting_jurisdiction='West Virginia' 
LIMIT
300000
"""

        
results = wclient.get("2ew6-ywp6", query=query)
res0=DataFrame.from_records(results)
print(res0.groupby('wwtp_id').nth(0))
print(res0[res0['wwtp_id'].isna()])
res0

In [ ]:
RSRC ='2ew6-ywp6'

# date>='2020-01-03T00:00:00.000' AND key_plot_id LIKE '%_{wstate}_%'
# date='2023-12-22T00:00:00.000'
query = f"""
SELECT
key_plot_id,
date_end,
percentile
WHERE
date_end>='2020-01-01T00:00:00.000'
LIMIT
700000
"""

results = wclient.get(RSRC, query=query)
res4=DataFrame.from_records(results)
res4['percentile'] =  res4['percentile'].astype(float)
res4['date_end'] = to_datetime(res4['date_end'])
res4=res4[res4['percentile'].notna()]


x=res4.groupby('date_end')['percentile'].count()
fig, ax = plt.subplots(1,1, figsize=(10,8))
ax.plot_date(x.index, x, ms=1)
ax.set_ylabel('# stations reporting')
format_axis(ax)
ax.tick_params('x', rotation=90)
fig.suptitle('Number wastewater stations reporting')
plt.savefig('num_waste_stations_per.png')

full_per = res4.join(grp.set_index('key_plot_id'), on='key_plot_id')
full_per.sort_values('date_end')
full_per=full_per.replace(999., 0.)

ngrp_per = full_per.groupby(['date_end']).apply(lambda x: x['percentile'].mean())
plt.figure()
plt.plot_date(ngrp_per.index, ngrp_per, ms=1)
plt.ylabel('concentration average')
plt.savefig('wastewater_by_nation_per.png')

sgrp_per = full_per.groupby(['date_end', 'wwtp_jurisdiction']).apply(lambda x: x['percentile'].mean()).reset_index(name='average concentration')
graph_perc_state = sbrn.FacetGrid(sgrp_per, col='wwtp_jurisdiction', col_order=get_col_order(sgrp, 'wwtp_jurisdiction'), sharex=True, sharey=False, col_wrap=8)
graph_perc_state.map(plt.plot_date, 'date_end', 'average concentration', ms=1)
graph_perc_state.tick_params('x', rotation=90)
graph_perc_state.savefig('wastewater_by_state_per.png')

In [ ]:
def _add_one(limit):
    one=0
    while one<limit:
        yield one
        one = one+1
add_one = _add_one(135)
plt.figure()
res4[res4['percentile']==999.].groupby('key_plot_id').apply(lambda x: plt.plot_date(x['date_end'], x['percentile']+next(add_one), ms=1))

In [ ]:
import plotly.express as px
fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])

import plotly.graph_objects as go
fig_widget = go.FigureWidget(fig)
fig_widget